# Cross-Classification Refinement 
For 2014 Survey data for 4k

Using a decision (classification) tree approach to update trip rates
Based on guidance from this report: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.539.3755&rep=rep1&type=pdf

Using python tools to build a decision tree for generating classifications
This process helps combine classes with small samle sizes to match existing crossclass definitions

In [4]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split

In [5]:
# Load household records to prepare data and generate cross-classification
hh = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\2014-pr3-M-hhsurvey-households.xlsx',
                  sheetname='Data')

# Reclassify household columns for cross-classification

# Number of Workers 0 -> 3+
hh['numworkers_crossclass'] = hh['numworkers']
hh.ix[hh['numworkers'] >= 3, 'numworkers_crossclass'] = 3
hh['numworkers_crossclass'] = hh['numworkers_crossclass'].astype('int')

# Household size
hh['hhsize_crossclass'] = hh['hhsize']
hh.ix[hh['hhsize'] >= 4, 'hhsize_crossclass'] = 4
hh['hhsize_crossclass'] = hh['hhsize_crossclass'].astype('int')

# Household income
# Note that exact ranges from 2014 do not match 2006
# old ranges: 30, 60, 90+
# new ranges: 35, 75, 100+
hh['income_crossclass'] = hh['hh_income_detailed_imp']
hh.ix[hh['hh_income_detailed_imp'] <= 3, 'income_crossclass'] = 1    # $35k  
hh.ix[(hh['hh_income_detailed_imp'] > 3) & (hh['hh_income_detailed_imp'] <= 5),    # $35-75k
           'income_crossclass'] = 2
hh.ix[(hh['hh_income_detailed_imp'] > 5) & (hh['hh_income_detailed_imp'] <= 6),    # $75-100k
           'income_crossclass'] = 3
hh.ix[hh['hh_income_detailed_imp'] >= 7, 'income_crossclass'] = 4    # >$100k

In [14]:
# Data needs to be in format of each household on a row and columns for trips by purpose
# Load trip data and compute 4k purposes

# 2014 File with adjustments from 2017 smartphone survey
trip = pd.read_excel(r'J:\Projects\Surveys\HHTravel\Survey2014\Data\Final database\Release 4\Adjusted\trips_2014_adjusted_wt.xlsx')

In [16]:
# Separate college student trips from regional survey trips
college_trips = trip[trip['hhid'] < 14000000]

# Filter out college student trips
trip = trip[trip['hhid'] >= 14000000]

# Filter out exercise and change mode trips
trip = trip[trip['d_purpose'] != 10]    
trip = trip[trip['d_purpose'] != 15]

# Home-Based Work (HBW) Trips, directly from home-to-work and work-to-home; for primary workplace only
trip.ix[(trip['o_purpose'] == 1) & (trip['d_purpose'] == 2), '4k_purp'] = 'HBW'
trip.ix[(trip['o_purpose'] == 2) & (trip['d_purpose'] == 1), '4k_purp'] = 'HBW'

# Home-Based shopping
trip.ix[(trip['o_purpose'] == 1) & (trip['d_purpose'].isin([4,5])), '4k_purp'] = 'Home-Based Shopping'
trip.ix[(trip['o_purpose'].isin([4,5])) & (trip['d_purpose'] == 1), '4k_purp'] = 'Home-Based Shopping'

# Home-Based School
trip.ix[(trip['o_purpose'] == 1) & (trip['d_purpose'] == 6), '4k_purp'] = 'School'
trip.ix[(trip['o_purpose'] == 6) & (trip['d_purpose'] == 1), '4k_purp'] = 'School'

# Home-Based Other
# Do not include exercise, change-mode trips
trip.ix[(trip['o_purpose'] == 1) & (trip['d_purpose'].isin([3,7,8,9,11,12,13,14,16])), '4k_purp'] = 'Home-Based Other'
trip.ix[(trip['o_purpose'].isin([3,7,8,9,11,12,13,14,16]) & (trip['d_purpose'] == 1)), '4k_purp'] = 'Home-Based Other'

# NHB Work-to-Other
trip.ix[(trip['o_purpose'] == 2) & (trip['d_purpose'] != 1) & (trip['4k_purp'].isnull()), '4k_purp'] = 'NHB WtO'
trip.ix[(trip['o_purpose'] != 1) & (trip['d_purpose'] == 2) & (trip['4k_purp'].isnull()), '4k_purp'] = 'NHB WtO'

# # NHB Other-to-Other (Destination and Origins are neither work nor home)
trip.ix[(trip['o_purpose'] != 2) & (trip['d_purpose'] != 1) & (trip['o_purpose'] != 1) 
        & (trip['d_purpose'] != 2) & (trip['4k_purp'].isnull()), '4k_purp'] = 'NHB OtO'

In [20]:
# Export GPS Trips with 4k_purp
trip.to_excel(r'R:\4K\2014\Trip Generation\Trip Rates\trip_2014_survey_adjusted.xlsx', index=False)

# Add household fields to data
# hh_fields = ['vehicle_count','hhsize','numadults','numchildren','numworkers','lifecycle',
#              'rent_own','res_type','income_crossclass','h_city','h_address','h_county_name','h_district_name',
#              'h_taz2010','hhid']
# export_trip_hh = pd.merge(trip, hh[hh_fields], left_on='hhno', right_on='hhid', how='left')
# export_trip_hh.to_csv(r'R:\4K\2014\Trip Generation\Trip Rates\trip_2014_survey_gps.csv', index=False)

In [23]:
# Apply a decision tree to figure out which rates should be averaged and combined

min_samples_size = 50
seed_value = 99
int_conversion = 1000

# Reorganize the trip record data
# Group by household id
_df = trip.groupby(['hhid','4k_purp']).sum()[['trip_wt_final']]
_df = _df.unstack()
_df.fillna(0,inplace=True)
_df = _df.reset_index()
purp_cols = ['HBW','Home-Based Other','Home-Based Shopping','NHB OtO','NHB WtO','Home-Based School']
_df.columns = ['hhid']+purp_cols

# # Join hh information 
df = pd.merge(_df, hh[['hhid','hhsize_crossclass','numworkers_crossclass', 'income_crossclass']], 
         on='hhid',how='left')

# Get total trips per household
df['tot_trips'] = df['HBW']+df['Home-Based Other']+df['Home-Based Shopping']+df['NHB OtO']+df['NHB WtO']+df['Home-Based School']

# For decision tree analysis, need a target (y) and features (x)
# We are modeling trips as a function of HH size, number of workers, and income classes

# Features for classification
features = ["hhsize_crossclass","numworkers_crossclass","income_crossclass"]
X = df[features]

# Decision trees are used to predict total trips
# since we are classifying based on household characteristics and not trips
# we can use total trips as dependent variable to generate bins

# Must be in integer form for prediction
# Convert to int by multiplying by factor
df['tot_trips_int'] = (df['tot_trips']*int_conversion).astype('int')

# Define the target y value as trips by purpsoe
y = df['tot_trips_int']

# Create training and testing sample (maybe not necessary...)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Create decision tree, set min sample size and random seed
# This part generates the optimal category separations
estimator = DecisionTreeClassifier(min_samples_leaf=min_samples_size, random_state=seed_value)

# Export for visualization
# visualize_tree(estimator, features)

# Use the new estimator object to assign categories to original data
estimator.fit(X_train, y_train)
new_categories = estimator.apply(X)
df['optimum_grouping'] = new_categories

In [24]:
# Group by old and new categories to apply new cross class blending
# For rows that have the same value for 'new_grouping' col, results for these should be averaged
df_cat = df.groupby(['optimum_grouping','hhsize_crossclass','income_crossclass','numworkers_crossclass']).count()

In [25]:
df_cat = df_cat.reset_index()

In [26]:
hh = pd.merge(hh, df_cat[['hhsize_crossclass','income_crossclass','numworkers_crossclass','optimum_grouping']], 
         how='left', on=['hhsize_crossclass','income_crossclass','numworkers_crossclass'])

In [29]:
# Total households by cross-classification group (hhsize, numworkers, and income)
total_hh = hh.groupby(['optimum_grouping']).sum()[['expwt_2']]
total_hh = total_hh.reset_index()

# Join the category values
total_hh = pd.merge(total_hh, df_cat[['hhsize_crossclass','numworkers_crossclass','income_crossclass','optimum_grouping']], 
                    how='left', on='optimum_grouping')
total_hh.to_clipboard()

In [30]:
hh.groupby(['optimum_grouping']).count()[['expwt_2']].to_clipboard()

In [32]:
# Calculate trips rates

# Load trip files
# GPS Weighted trips (in Daysim format)


# Merge household grouping to trip
trip_hh = pd.merge(trip, hh, on='hhid', how='left')

In [37]:
# Trips by Purpose
# purp = 'HBW'
# purp = 'Home-Based Shopping'
# purp = 'Home-Based Other'
# purp = 'School'
purp = 'NHB OtO'
# purp = 'NHB WtO'
_df = trip_hh[trip_hh['4k_purp'] == purp].groupby(['optimum_grouping']).sum()[['trip_wt_final']]
_df = _df.reset_index()
_df = pd.merge(_df, df_cat[['hhsize_crossclass','numworkers_crossclass','income_crossclass','optimum_grouping']], 
                    how='left', on='optimum_grouping')
_df.to_clipboard()

# Get trip totals by purpose
# trip_hh[trip_hh['4k_purp'] == purp]['trexpfac'].sum()

In [39]:
# Get trip totals
trip_hh.groupby('4k_purp').sum()['trip_wt_final']

4k_purp
HBW                    2.339437e+06
Home-Based Other       3.944246e+06
Home-Based Shopping    1.383019e+06
NHB OtO                2.504022e+06
NHB WtO                1.270762e+06
School                 1.291760e+06
Name: trip_wt_final, dtype: float64

In [65]:
trip_hh.groupby('4k_purp').sum()['trexpfac'].sum()

12960491.149999935